<a href="https://colab.research.google.com/github/cristinarainich/Sternberg_analysis/blob/main/Environmental.%20Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io
from pathlib import Path

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental')

In [4]:
!ls /content/gdrive/MyDrive/CoganLab/Environmental

 D29_Block_1_TrialData.mat	 'D77_Block_1_TrialData (1).mat'
 D64_Block_1_TrialData.mat	  eeg_envir.csv
 D65_Block_1_TrialData_1.mat	  Environmental_Subject.mat
 D65_Block_1_TrialData.mat	  healthy
'D71_Block_1_TrialData (1).mat'   ieeg_envir.csv
'D73_Block_1_TrialData (2).mat'   stimuli


In [5]:
# Patients
ieeg = scipy.io.loadmat(
    working_path.joinpath('Environmental_Subject.mat')
    )

subj1 = scipy.io.loadmat(
    working_path.joinpath('D65_Block_1_TrialData.mat')
    )

subj65 = scipy.io.loadmat(
    working_path.joinpath('D65_Block_1_TrialData_1.mat')
    )

subj64 = scipy.io.loadmat(
    working_path.joinpath('D64_Block_1_TrialData.mat')
)
subj73 = scipy.io.loadmat(
    working_path.joinpath('D73_Block_1_TrialData (2).mat')
    )
subj77 = scipy.io.loadmat(
    working_path.joinpath('D77_Block_1_TrialData (1).mat')
    )

In [6]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental/healthy')

In [7]:
# healthy
files = [] 
begin = 'E'
end = '_Block_1_TrialData.mat'
files.append(begin + '32' + end)
for i in range(34, 47):
  files.append(begin + str(i) + end)

In [8]:
files[0]

'E32_Block_1_TrialData.mat'

In [9]:
eeg = []
for i in files:
  subset = scipy.io.loadmat(
      working_path.joinpath(i)
  )
  eeg.append(subset)

#Working with stimuli

In [144]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental/stimuli')

In [145]:
!ls /content/gdrive/MyDrive/CoganLab/Environmental/stimuli

imaginability.txt	 proposed_nw.mat	 proposed_w3.mat  untrn_w.mat
proposed_nw3_global.mat  proposed_w2.mat	 proposed_w.mat
proposed_nw3.mat	 proposed_w3_global.mat  untrn_nw.mat


In [146]:
stimuli_words = scipy.io.loadmat(
    working_path.joinpath('proposed_w3.mat')
)

In [147]:
columns = stimuli_words['proposed_w'].dtype.names
stim_words = pd.DataFrame(columns=columns)
for i in range(0, stimuli_words['proposed_w'].shape[1]):
  values = []
  subset = stimuli_words['proposed_w'][0][i]
  for name in columns:
    values.append(subset[name])
  stim_words.loc[len(stim_words.index)] = values

In [148]:
stimuli = []
for i in stim_words.Word.values:
  stimuli.append(i[0])

freq = []
for i in stim_words.SFreq.values:
  freq.append(i[0][0])

imagine = []
for i in stim_words.imaginability.values:
  imagine.append(i[0][0])

category = []
for i in range(0, 24):
  category.append('word')

In [149]:
columns = ['Category', 'Stimulus', 'Frequency', 'Imaginability']
words = pd.DataFrame(columns=columns)
words['Category'] = category
words['Stimulus'] = stimuli
words['Frequency'] = freq
words['Imaginability'] = imagine

In [150]:
stimuli_nonwords = scipy.io.loadmat(
    working_path.joinpath('proposed_nw3.mat')
)

In [151]:
columns = stimuli_nonwords['proposed_nw'].dtype.names
stim_nonwords = pd.DataFrame(columns=columns)
for i in range(0, stimuli_nonwords['proposed_nw'].shape[1]):
  values = []
  subset = stimuli_nonwords['proposed_nw'][0][i]
  for name in columns:
    values.append(subset[name])
  stim_nonwords.loc[len(stim_nonwords.index)] = values

In [152]:
stimuli = []
for i in stim_nonwords.Word.values:
  stimuli.append(i[0])

freq = []
for i in stim_nonwords.SFreq.values:
  freq.append(i[0][0])

category = []
for i in range(0, 24):
  category.append('nonword')

In [153]:
columns = ['Category', 'Stimulus', 'Frequency', 'Imaginability']
nonwords = pd.DataFrame(columns=columns)
nonwords['Category'] = category
nonwords['Stimulus'] = stimuli
nonwords['Frequency'] = freq
nonwords['Imaginability'] = np.nan

In [154]:
all_stim = pd.concat([words, nonwords])

# iEEG dataset

In [ ]:
data_ieeg = ieeg['Subject']
print('Number of subjects in EEG dataset is', data_ieeg.shape[1])

Number of subjects in EEG dataset is 19


In [ ]:
subjects_ieeg = [] # subject numbers
for i in data_ieeg['Name'][0]:
  subjects_ieeg.append(i[0])
print(subjects_ieeg)

['D27', 'D28', 'D30', 'D31', 'D32', 'D35', 'D37', 'D38', 'D39', 'D41', 'D42', 'D47', 'D53', 'D54', 'D55', 'D57', 'D63', 'D69', 'D71']


In [ ]:
columns = []
for i in range(0, data_ieeg.shape[1]):
  actual = data_ieeg[0][i]['trialInfo'][0][0].dtype.names
  for name in actual:
    if name not in columns:
      columns.append(name)

In [ ]:
def get_to_df(data):
  df = pd.DataFrame(columns=columns)
  df['Trial'] = np.nan
  df['Subject'] = np.nan

  for i in range(0, data.shape[1]):
    # for each subject
    subset = data[0][i]
    subject = subset['Name'][0]
    print(subset['Task'])
    print(subject)
    for trial in range(0, subset['trialInfo'].shape[1]):
      # for each trial
      sub = subset['trialInfo'][0][trial]
      dtype = sub.dtype.names
      values = []
      for name in columns:
        if name in dtype:
          values.append(sub[name])
        else:
          values.append(np.nan)
      values.append(str(trial)) # adding a trial number
      values.append(subject)
      df.loc[len(df.index)] = values
  return df

In [ ]:
ieeg = get_to_df(data_ieeg)

['Environmental_Sternberg']
D27
['Environmental_Sternberg']
D28
['Environmental_Sternberg']
D30
['Environmental_Sternberg']
D31
['Environmental_Sternberg']
D32
['Environmental_Sternberg']
D35
['Environmental_Sternberg']
D37
['Environmental_Sternberg']
D38
['Environmental_Sternberg']
D39
['Environmental_Sternberg']
D41
['Environmental_Sternberg']
D42
['Environmental_Sternberg']
D47
['Environmental_Sternberg']
D53
['Environmental_Sternberg']
D54
['Environmental_Sternberg']
D55
['Environmental_Sternberg']
D57
['Environmental_Sternberg']
D63
['Environmental_Sternberg']
D69
[]
D71


In [ ]:
ieeg.groupby(['Subject'])['Trial'].count()

Subject
D27    160
D28    120
D30    168
D31    168
D32    168
D35    168
D37    168
D38    160
D39    168
D41    168
D42    168
D47    168
D53    168
D54    168
D55    168
D57    168
D63    168
D69    168
D71    168
Name: Trial, dtype: int64

In [ ]:
# adding subjects D64
df_64 = pd.DataFrame(columns=columns)
df_64['Trial'] = np.nan
df_64['Subject'] = np.nan
for trial in range(0, subj2['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj2['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D64')
  df_64.loc[len(df_64.index)] = values

In [ ]:
# adding subjects D65
df_65 = pd.DataFrame(columns=columns)
df_65['Trial'] = np.nan
df_65['Subject'] = np.nan
for trial in range(0, subj1['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj2['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D65')
  df_65.loc[len(df_65.index)] = values
  i = 96
for trial in range(0, subj1_1['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj2['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(i)) # adding a trial number
  values.append('D65')
  i += 1
  df_65.loc[len(df_65.index)] = values

In [ ]:
ieeg = pd.concat([ieeg, df_64])
ieeg = pd.concat([ieeg, df_65])

In [ ]:
# adding subject 73
df_73 = pd.DataFrame(columns=columns)
df_73['Trial'] = np.nan
df_73['Subject'] = np.nan
for trial in range(0, subj73['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj73['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D73')
  df_73.loc[len(df_73.index)] = values

In [ ]:
# adding subject 77
df_77 = pd.DataFrame(columns=columns)
df_77['Trial'] = np.nan
df_77['Subject'] = np.nan
for trial in range(0, subj77['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj77['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D77')
  df_77.loc[len(df_77.index)] = values

In [ ]:
ieeg = pd.concat([ieeg, df_73])
ieeg = pd.concat([ieeg, df_77])

In [ ]:
ieeg.groupby(['Subject'])['Trial'].count()

Subject
D27    160
D28    120
D30    168
D31    168
D32    168
D35    168
D37    168
D38    160
D39    168
D41    168
D42    168
D47    168
D53    168
D54    168
D55    168
D57    168
D63    168
D64    168
D65    168
D69    168
D71    168
D73    168
D77    168
Name: Trial, dtype: int64

# EEG dataset

In [10]:
subjects = []
for i in files:
  subjects.append(i[:3])

In [11]:
columns = []
for subject in eeg:
  # for each subject
  actual = subject['trialInfo'][0][0].dtype.names
  for name in actual:
    if name not in columns:
      columns.append(name)

In [12]:
df_ = pd.DataFrame(columns=columns)
df_['Trial'] = np.nan
df_['Subject'] = np.nan
for subject in eeg:
  for trial in range(0, subject['trialInfo'].shape[1]):
    # for each trial
    values = []
    subset = subject['trialInfo'][0][trial]
    dtype = subset.dtype.names
    for name in columns:
      if name in dtype:
        values.append(subset[name])
      else:
        print('check')
        values.append(np.nan)
    values.append(str(trial))
    values.append(subjects[eeg.index(subject)])
    df_.loc[len(df_.index)] = values

In [13]:
eeg = df_.copy()

In [14]:
eeg.groupby(['Subject'])['Trial'].count()

Subject
E32    216
E34    216
E35    216
E36    216
E37    216
E38    216
E39    216
E40    184
E41    216
E42    216
E43    216
E44    216
E45    216
E46    216
Name: Trial, dtype: int64

#Preprocessing EEG

In [15]:
eeg.Subject.unique()

array(['E32', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40', 'E41',
       'E42', 'E43', 'E44', 'E45', 'E46'], dtype=object)

In [72]:
eeg_ = eeg[[
    'block', 'StimulusCategory', 'ReactionTime',
       'Resp', 'RespCorrect', 'stimulusSounds_idx',
       'ProbeCategory', 'probeSound_idx',
       'Trial', 'Subject', 'stimulusSounds_name', 'probeSound_name'
]]

In [17]:
len(eeg.Subject.unique())

14

Getting rid of additional lists containing lists

In [74]:
block = eeg_.block.values
new_block = []
for i in block:
  while type(i) is np.ndarray:
    i = i[0]
  new_block.append(i)

In [75]:
stim_cat = eeg_.StimulusCategory.values
new_stim = []
for i in stim_cat:
  if 'environment' in i:
    new_stim.append('environment')
  elif 'nonwords' in i:
    new_stim.append('nonwords')
  elif 'words' in i:
    new_stim.append('words')

In [76]:
rt = eeg_.ReactionTime.values
new_rt = []
for i in rt:
  while type(i) is np.ndarray:
    i = i[0]
  new_rt.append(i)

In [77]:
acc = eeg_.RespCorrect.values
new_acc = []
for i in acc:
  while type(i) is np.ndarray:
    i = i[0]
  new_acc.append(i)

In [78]:
resp = eeg_.Resp.values
new_resp = []
for i in resp:
  while type(i) is np.ndarray:
    i = i[0]
  new_resp.append(i)

In [79]:
probe = eeg_.ProbeCategory.values
new_probe = []
for i in probe:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe.append(i)

In [80]:
probe_idx = eeg_.probeSound_idx.values
new_probe_idx = []
for i in probe_idx:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_idx.append(i)

In [100]:
stim_idx = eeg_.stimulusSounds_idx.values
new_stim_idx = []
for i in stim_idx:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0])
  new_stim_idx.append(element)

In [102]:
eeg_['block'] = new_block
eeg_['StimulusCategory'] = new_stim
eeg_['ReactionTime'] = new_rt
eeg_['RespCorrect'] = new_acc
eeg_['Resp'] = new_resp
eeg_['ProbeCategory'] = new_probe
eeg_['probeSound_idx'] = new_probe_idx
eeg_['stimulusSounds_idx'] = new_stim_idx

<ipython-input-102-ea534a877bb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['block'] = new_block
<ipython-input-102-ea534a877bb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['StimulusCategory'] = new_stim
<ipython-input-102-ea534a877bb7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [104]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  if len(i) not in length:
    length.append(len(i))
print(length)

[3, 7, 9, 5]


In [105]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  length.append(len(i))
eeg_['Length'] = length

<ipython-input-105-d50296b58efb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Length'] = length


In [106]:
words = []
nonwords = []
environment = []
for i in eeg_.StimulusCategory.values:
  if i == 'nonwords':
    words.append(0)
    nonwords.append(1)
    environment.append(0)
  elif i == 'words':
    words.append(1)
    nonwords.append(0)
    environment.append(0)
  elif i == 'environment':
    words.append(0)
    nonwords.append(0)
    environment.append(1)

In [107]:
eeg_['Words'] = words
eeg_['Nonwords'] = nonwords
eeg_['Environment'] = environment

<ipython-input-107-3c31c9b484ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Words'] = words
<ipython-input-107-3c31c9b484ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Nonwords'] = nonwords
<ipython-input-107-3c31c9b484ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [130]:
probe_names = eeg_.probeSound_name.values
new_probe_names = []
for name in probe_names:
  name = name[0][0][0]
  new_probe_names.append(name)

In [140]:
stimulus_names = eeg_.stimulusSounds_name.values
new_stim_names = []
for name in stimulus_names:
  name = name[0][0]
  new_name = []
  for j in name:
    new_name.append(j[0][0])
  new_stim_names.append(new_name)

In [195]:
new_new_stim_names = []
for element in new_stim_names:
  new_element = []
  for name in element:
    new_element.append(name[:-4])
  new_new_stim_names.append(new_element)

In [192]:
new_new_probe_names = []
for i in new_probe_names:
  new_new_probe_names.append(i[:-4])

In [197]:
eeg_['stimulusSounds_name'] = new_new_stim_names
eeg_['probeSound_name'] = new_new_probe_names

<ipython-input-197-a747f04adf7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['stimulusSounds_name'] = new_new_stim_names
<ipython-input-197-a747f04adf7e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['probeSound_name'] = new_new_probe_names


**Adding frequency and imaginabilty scores**

In [209]:
category = eeg_.StimulusCategory.values.tolist()
probe = eeg_.probeSound_name.values
stimuli = eeg_.stimulusSounds_name.values
standard = all_stim[all_stim['Category'] == 'word']['Stimulus'].values.tolist()
standard_freq = all_stim[all_stim['Category'] == 'word']['Frequency'].values.tolist()
standard_imag = all_stim[all_stim['Category'] == 'word']['Imaginability'].values.tolist()

stim_freq = []
stim_imag = []
probe_freq = []
probe_imag = []

for i in range(0, eeg_.shape[0]):
  if category[i] == 'words':
    for name in standard:
      if probe[i] == name:
        freq = standard_freq[standard.index(name)]
        imag = standard_imag[standard.index(name)]
        probe_freq.append(freq)
        probe_imag.append(imag)

    values_freq = []
    values_imag = []      
    for lis in stimuli[i]:
      for name in standard:
        if lis == name:
          freq = standard_freq[standard.index(name)]
          imag = standard_imag[standard.index(name)]
          values_freq.append(freq)
          values_imag.append(imag)
    stim_freq.append(values_freq)
    stim_imag.append(values_imag)
  
  else:
    stim_freq.append(np.nan)
    stim_imag.append(np.nan)
    probe_freq.append(np.nan)
    probe_imag.append(np.nan)

In [210]:
print(eeg_.shape)
print(len(stim_freq))
print(len(stim_imag))
print(len(probe_freq))
print(len(probe_imag))

(2992, 16)
2992
2992
2992
2992


In [211]:
eeg_['ProbeFreq'] = probe_freq
eeg_['ProbeImag'] = probe_imag
eeg_['StimFreq'] = stim_freq
eeg_['StimImag'] = stim_imag

<ipython-input-211-c1691c97d5de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['ProbeFreq'] = probe_freq
<ipython-input-211-c1691c97d5de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['ProbeImag'] = probe_imag
<ipython-input-211-c1691c97d5de>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

# Saving files

In [213]:
working_path = 

PosixPath('/content/gdrive/MyDrive/CoganLab/Environmental/stimuli')

In [ ]:
eeg.to_csv(
    Path(working_path).joinpath('eeg_envir.csv')
)

In [ ]:
ieeg.to_csv(
    Path(working_path).joinpath('ieeg_envir.csv')
)